Strings & Files 1.4 Workshop Submission:
Sasha Morrison, (QuantFreeman)

# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [1]:
d = {1 : 2, 3 : 4}

def safe_dict(d, k):
    
    return ( 
        d.get(k) if k in d     
        else 0 )   

safe_dict(d, 'cat')

0

In [2]:
# Alternate using a lambda function:

d = {1 : 2, 3 : 4}

safe_dict = lambda d, k : d.get(k) if k in d else 0

safe_dict(d, 'cat')

0

# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentioned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [21]:
def hamcount():
    out = []
    counter = 0
    hamlet = open('data/hamlet.txt', 'r')
    read = hamlet.read()
    split = read.split("\\n")
    # split text at line breaks into a list of strings
    for e in split:
        ap = e.lower().split()
        # cast to lowercase and split by blank space
        out.extend(ap)
    for e in out:
        if 'hamlet' in e:
            counter += 1
    return counter
hamcount()

474

### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [22]:
with open('hamcount.py', 'w') as f:
    f.write(
'''
def hamcount():
    out = []
    counter = 0
    hamlet = open('data/hamlet.txt', 'r')
    read = hamlet.read()
    split = read.split('\\n')
    # split text at line breaks into a list of strings
    for e in split:
        ap = e.lower().split()
        # cast to lowercase and split by blank space
        out.extend(ap)
    for e in out:
        if 'hamlet' in e:
            counter += 1
    return counter
''')
    

In [23]:
import hamcount as ham
ham.hamcount()

474

### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [80]:
punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~â€”™ï»¿'
punc2 = '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~”™ï»¿'
out = []
hamlet = open('data/hamlet.txt', 'r')
read = hamlet.read()
split = read.split('\n')
# split text at line breaks into a list of strings
for e in split:
    ap = e.lower().split()
    # cast to lowercase and split by blank space
    out.extend(ap)

#these two transform some weird chars into proper chars
for e in out:
    if 'â€™' in e:
        pos = e.find('â€™')
        start = pos
        end = start + 3
        out[out.index(e)] = e[0:start] + "'" + e[end:]
        
for e in out:
    if 'â€™' in e:
        pos = e.find('â€™')
        start = pos
        end = start + 3
        out[out.index(e)] = e[0:start] + "'" + e[end:]
        
for e in out:        
        if 'â€”' in e:
            pos = e.find('â€”')
            start = pos
            end = start + 3
            out[out.index(e)] = e[0:start] + "-" + e[end:]

# this is trimming punc on the start or end of each word, repeats 4x cause it only does 1 at a time
for i in range(4):
    for e in out:
        if e[0] in punc:
            out[out.index(e)] = e[1:]

    for e in out:
        if e[-1] in punc:
            out[out.index(e)] = e[:-1]

# this is trimming out instances of 2 consecutive punctuations
for e in out:
    flag = 0
    flag2 = 0
    for i in range(len(e)):
        if flag == 1:
            if e[i] in punc2:
                upd = str(e[0:(i-1)] + ' ' + e[(i+1):])
                upd = upd.split()
                out.extend(upd)
                flag = 0
                flag2 = 1
            else: 
                flag = 0
                
        if flag == 0:
            if e[i] in punc2:
                flag = 1
    if flag2 == 1:
        out.pop(out.index(e))
        
print('unique words in hamlet, rough estimate: ' + str(len(set(out))))
# set(out)

unique words in hamlet, rough estimate: 4851


{'slander',
 'overthrown',
 'marcellus',
 'cannot',
 'tempest',
 'feeling',
 'caviare',
 "fear'd",
 'bloat',
 'dreadfully',
 'alack',
 'freely',
 'countries',
 'moor',
 'deal',
 "man's",
 "soul's",
 'commerce',
 'along',
 'vain',
 'paris',
 'post',
 'bands',
 'withers',
 'cataplasm',
 'closet',
 'holds',
 'abridgement',
 'touching',
 'envy',
 'says',
 'burden',
 'partisan',
 "o'erhanging",
 'forget',
 'sledded',
 'rough-hew',
 'fiends',
 'reynaldo',
 'content',
 'pith',
 'aroused',
 'jephthah',
 'smelt',
 'exhort',
 'precisely',
 'sullies',
 'rack',
 'never',
 'knows',
 'slanders',
 'motion',
 'spades',
 'neighbour',
 'hurt',
 'wishes',
 "t'other",
 'calamity',
 'pay',
 'gratis',
 'flatter',
 'you',
 'treasure',
 'trip',
 'nations',
 "braz'd",
 'vigour',
 'feast',
 'scarce',
 'haunt',
 'conveyances',
 'loneliness',
 'a-work',
 "restrain'd",
 'weapons',
 "liv'd",
 'sending',
 "barber's",
 'confront',
 'villanies',
 "devis'd",
 "devotion's",
 'southerly',
 'grows-the',
 'goodly',
 'forts

# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [14]:
import os

def file_count(inp):
    count = 0
    files = os.listdir(inp)
    
    for f in files:
        if f[-3:] == '.py':
            count += 1
    return count

file_count('data/csrgraph')

8

### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [15]:
import os

def pack_count(inp):
    
    package_list = {'pandas':0, 'numpy':0, 'numba':0 }
    # depending on desired specificity, can change to 'import pandas/numpy/numba' 
    # this lowers count of numpy where 'from numpy import jit' etc
    files = os.listdir(inp)
    for f in files:
        try:
            file = inp + '/' + f
            current = open(file, 'r')
            read = current.read()
            for k in package_list:
                if k in read:
                    package_list[k] += 1
        except: FileNotFoundError
        # currently using a try/except for the file not found error 
        # which occurs when an '.ipynb_checkpoints' file is found in listdir
    return package_list

pack_count('data/csrgraph')

{'pandas': 4, 'numpy': 6, 'numba': 6}

# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

In [16]:
"""
log of ( N (total n of docs) / 1 + n(w) )
(where n(w) is number of docs where given token occurs) 
given this formula, i create a set from the doc before parsing 
in order to eliminate reptitions so that:
['interview', 'questions', 'interview'] produces -0.4, as per the formula
"""

import numpy as np

def idf(docs):
    N = len(docs)
    dic = {}
    out = {}
    nw = 0
    
    for doc in docs:
        setdoc = set(doc)
        for tok in setdoc:
#             dic[tok] = dic.get(tok, 0) + 1
#             this method was suggested by Jasleen 
#             but I don't quite understand it yet
            if tok not in dic:
                dic[tok] = 1
            elif tok in dic:
                dic[tok] += 1
    
    for k in dic:
        nw = dic.get(k)
        out[k] = np.log( N / (1 + nw) )   
    return out
        
idf([['interview', 'questions', 'interview'], ['interview', 'answers']])

{'questions': 0.0, 'interview': -0.40546510810816444, 'answers': 0.0}

# Stretch Goal: TF-IDF on Hamlet

The TF-IDF score is a commonly used statistic for the importance of words. Its $\frac{TF}{IDF}$ where TF is the "term frequency" (eg. how often the words happens in the document).

Calculate the TF-IDF dictionary on the Hamlet book.

What's the TF-IDF of "Hamlet"?

What's the word with the highest TF-IDF in the book?